In [ ]:
pip install tensorflow_text

In [3]:
pip install scikeras

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install bayesian-optimization==1.4.1

Reason for being yanked: https://github.com/fmfn/BayesianOptimization/pull/388
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
import tensorflow_hub as hub
import tensorflow as tf
from tensorflow.keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout, Input, Concatenate, Conv1D, GlobalMaxPooling1D, Flatten, Lambda, Reshape
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
from scikeras.wrappers import KerasClassifier
from math import floor
from sklearn.metrics import make_scorer, accuracy_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold

from keras.layers import LeakyReLU
LeakyReLu = LeakyReLU(alpha=0.1)
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

2024-08-06 12:27:24.217956: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-06 12:27:24.325081: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-06 12:27:24.375563: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-06 12:27:24.389425: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-06 12:27:24.481118: I tensorflow/core/platform/cpu_feature_guar

In [2]:
from tqdm import tqdm

# Retrieve Data

In [3]:
data = pd.read_csv('updated_data.csv')
data.head()

,tweet_text,retweet_count,favorite_count,followers_count,friends_count,verified,label,time_delay (min),neg_sent,pos_sent,neu_sent,comp_sent,difference,weight,ratio
0,"Reports of ""moving body"" amidst #Germanwings w...",38,15,337960,6384,1,1,1.2833,0.000,0.00,1.000,0.0000,0.0,0.0,1.000000e+00
1,BREAKING:148passengers were on board #GermanWi...,43,15,52815,293,0,1,1.5000,0.000,0.14,0.860,0.3818,1.0,1.0,1.000000e+10
2,BREAKING: #Germanwings crash victims include 7...,31,5,893549,2312,1,1,4.1333,0.278,0.00,0.722,-0.6124,-0.2,-0.2,6.666667e-01
3,BREAKING: 148 feared dead in crashed #Germanwi...,167,32,418641,1859,1,1,3.5167,0.517,0.00,0.483,-0.8176,0.0,0.0,1.000000e+00
4,Terrible news... Airbus A320 from Barcelona to...,26,1,11062,233,0,1,3.3667,0.171,0.00,0.829,-0.4767,0.0,0.0,1.000000e+00


In [4]:
data.isna().sum()

tweet_text          0
retweet_count       0
favorite_count      0
followers_count     0
friends_count       0
verified            0
label               0
time_delay (min)    0
neg_sent            0
pos_sent            0
neu_sent            0
comp_sent           0
difference          0
weight              0
ratio               0
dtype: int64

# Split data

## All Features

In [5]:
text_data = data['tweet_text'].values
numerical_data = data[['retweet_count', 'followers_count', 'verified',
                           'time_delay (min)', 'neg_sent', 'pos_sent', 
                       'neu_sent', 'comp_sent', 'difference', 'ratio']].values
y = data['label']

In [6]:
# we added stratifying sampling
X_train_text, X_test_text, X_train_num, X_test_num, y_train, y_test = train_test_split(text_data, numerical_data, y, 
                                                                                       test_size=0.2, 
                                                                                       random_state=42, stratify=y)

In [7]:
# Get the shape of all the datasets
print(f'Train text: {len(X_train_text)}'), print(f'Train number: {len(X_train_num)}')
print(f'Test text: {len(X_test_text)}'), print(f'Test number:{len(X_test_num)}')
print(f'Y Training: {len(y_train)}'), print(f'Y Testing:{len(y_test)}')

Train text: 4641
Train number: 4641
Test text: 1161
Test number:1161
Y Training: 4641
Y Testing:1161


(None, None)

# Preprocessing

## Normalizing text features

In [10]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # Use CPU only

In [11]:
# Load ELMo from Tensorflow
elmo = hub.load("https://tfhub.dev/google/elmo/3")

In [12]:
#method to create word embeddings using ELMo 
def elmo_embeddings(text_list):
    embeddings = elmo.signatures['default'](tf.constant(text_list))['elmo']
    return embeddings

In [13]:
def process_in_batches(texts, batch_size=32):
    elmo_embed_np = []
    total_batches = (len(texts) + batch_size - 1) // batch_size  # Calculate total number of batches
    with tqdm(total=total_batches, desc='Processing Batches') as pbar:
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i + batch_size]
            batch_embeddings = elmo_embeddings(batch_texts)
            # Average the embeddings for each text
            batch_embeddings_np = [np.mean(embed.numpy(), axis=0) for embed in batch_embeddings]
            for embed in batch_embeddings_np:
                if embed.shape != (1024,):
                    print(f"Unexpected shape found: {embed.shape}")
            elmo_embed_np.extend(batch_embeddings_np)
            pbar.update(1)  # Update progress bar
    return np.array(elmo_embed_np)

In [14]:
# Process ELMo embeddings in batches
train_elmo_embed_np = process_in_batches(X_train_text, batch_size=16)  # Adjust batch size as needed
test_elmo_embed_np = process_in_batches(X_test_text, batch_size=16)  # Adjust batch size as needed

Processing Batches: 100%|███████████████████████| 73/73 [00:57<00:00,  1.27it/s]


In [18]:
print(type(train_elmo_embed_np))

<class 'numpy.ndarray'>


In [19]:
train_elmo_embed_np.shape

(4641, 1024)

In [20]:
print(f'Minimum value of training: {train_elmo_embed_np.min()}')
print(f'Maximum value of training: {train_elmo_embed_np.max()}')

Minimum value of training: -2.2559733390808105
Maximum value of training: 2.334711790084839


In [21]:
test_elmo_embed_np.shape

(1161, 1024)

## Normalizing Numerical features

In [16]:
from sklearn.preprocessing import MinMaxScaler

In [15]:
#using custom range
class CustomMinMaxScaler:
    def __init__(self, feature_range=(-2, 2)):
        self.feature_range = feature_range
        self.scaler = MinMaxScaler(feature_range=(0, 1))
    
    def fit(self, X):
        # Fit the scaler to the data
        self.scaler.fit(X)
        return self
    
    def transform(self, X):
        # Transform the data to [0, 1] range
        X_normalized = self.scaler.transform(X)
        # Scale to the desired range [-2, 2]
        a, b = self.feature_range
        X_scaled = a + (X_normalized * (b - a))
        return X_scaled
    
    def fit_transform(self, X):
        # Fit and transform the data
        return self.fit(X).transform(X)

In [17]:
num_scaler = CustomMinMaxScaler(feature_range=(-2, 2))

train_norm_num_features = num_scaler.fit_transform(X_train_num)
test_norm_num_features = num_scaler.fit_transform(X_test_num)
print(f'Normalized test shape: {test_norm_num_features.shape}')

Normalized test shape: (1161, 10)


In [18]:
print(f'Minimum value of training: {train_norm_num_features.min()}')
print(f'Maximum value of training: {test_norm_num_features.max()}')

Minimum value of training: -2.0
Maximum value of training: 2.0


## Concatenate Normalized features

In [19]:
X_train = np.hstack((train_elmo_embed_np, train_norm_num_features))
X_test = np.hstack((test_elmo_embed_np, test_norm_num_features))

print(f'Train shape: {X_train.shape}')
print(f'Test shape: {X_test.shape}')

Train shape: (4641, 1034)
Test shape: (1161, 1034)


In [20]:
def cnn_model(neurons, learning_rate, batch_size,epochs,
                  conv_layers, conv_filters, conv_kernel_size, 
                  dense_layers, normalization, dropout, dropout_rate):

    ## build model architecture
    input_x = Input(shape=(1034,))

    ## Reshape to fit Conv1D input
    x = Reshape((input_x.shape[1], 1))(input_x)

    ## Define Convolutional Layers
    for i in range(conv_layers):
        x = Conv1D(filters=conv_filters, kernel_size=conv_kernel_size, activation='relu')(x)
        if normalization > 0.5:
            x = BatchNormalization()(x)
                
    ## apply pooling to the given output x
    x = MaxPooling1D(pool_size=pool_size)(x)

    ## flatten the layer 
    x = Flatten()(x)
        
    ## Define dense layers
    for i in range(dense_layers):
        x = Dense(neurons, activation='relu')(x)
    
        if dropout>0.5:
            x = Dropout(dropout_rate, seed=123)(x)

    ## Define an output layer where the final prediction is made
    output = Dense(1, activation='sigmoid')(x)
        
    ## Define the model
    model = Model(inputs=input_x, outputs = output)
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate = learning_rate),
                      metrics=['accuracy', 'recall', 'Precision', 'FalseNegatives'])
    return model

# Grid search

In [21]:
input_shape = X_train.shape
print(input_shape)

(4641, 1034)


In [22]:
param_grid = {
    'batch_size': [16, 32],
    'epochs': [10, 20],
    'optimizer': ['adam', 'sgd'],
    'filters': [32, 64],
    'kernel_size': [3, 5],
    'pool_size': [2, 3],
    'activation': ['relu', 'tanh']
}

In [23]:
# Define the CNN model
def cnn_model(lr=0.001, filters=64, kernel_size=3, pool_size=2,
              num_conv_layers = 1, num_dense_layers=1, rate=0.5, units=64):
    model = Sequential()
    # Add convolutional layers
    for _ in range(num_conv_layers):
        model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', padding='same', input_shape=(1034, 1)))
        model.add(MaxPooling1D(pool_size=pool_size))
    
    model.add(Flatten())
    for _ in range(num_dense_layers):
        model.add(Dense(units=units, activation='relu'))
        model.add(Dropout(rate))
        
    model.add(Dense(1, activation='sigmoid'))  # Assuming binary classification
    
    model.compile(optimizer=Adam(learning_rate = lr), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [24]:
from bayes_opt import BayesianOptimization
from scikeras.wrappers import KerasClassifier
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from keras.layers import MaxPooling1D
# Define KerasClassifier with build_fn
model = KerasClassifier(build_fn=cnn_model, verbose=0, lr = [0.0001, 0.001, 0.01, 0.1], filters = [32, 64, 128],
                        kernel_size = [3, 5], pool_size = [2, 3], num_conv_layers = [1, 2, 3], 
                        num_dense_layers = [1, 2, 3], rate = [0.2, 0.3, 0.4, 0.5], units = [64, 128, 256])

# Define the grid search parameters
lr = [0.001, 0.01]
batch_size = [16, 32]
epochs = [10, 20, 30]
filters = [32, 64]
kernel_size = [3, 5]
pool_size = [2, 3]
num_conv_layers = [1, 2, 3]
num_dense_layers = [1, 2, 3]
rate = [0.3, 0.4, 0.5]
units = [64, 128]
# Define the search space for Bayesian optimization
# Define the grid search parameters
param_grid = dict(
    lr=lr,
    batch_size=batch_size,
    epochs=epochs,
    filters=filters,
    kernel_size=kernel_size,
    pool_size=pool_size,
    num_conv_layers=num_conv_layers,
    num_dense_layers=num_dense_layers,
    rate=rate,
    units=units
)
# Setup BayesSearchCV
opt = BayesSearchCV(
    estimator=model,
    search_spaces=param_grid,
    n_iter=10,  # Number of iterations for Bayesian optimization
    cv=10,  # Cross-validation folds
    verbose=1
)

# Fit the model
opt.fit(X_train, y_train)
opt.best_params_

Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits


OrderedDict([('batch_size', 26),
             ('epochs', 30),
             ('filters', 44),
             ('kernel_size', 4),
             ('lr', 0.0017702277771397554),
             ('num_conv_layers', 1),
             ('num_dense_layers', 1),
             ('pool_size', 2),
             ('rate', 0.5),
             ('units', 94)])

In [34]:
# Define the CNN model
def cnn_model(lr, filters, kernel_size, pool_size,num_conv_layers, 
              num_dense_layers, rate, units):
    model = Sequential()
    # Add convolutional layers
    for _ in range(num_conv_layers):
        model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', padding='same', input_shape=(1034, 1)))
        model.add(MaxPooling1D(pool_size=pool_size))
    
    model.add(Flatten())
    for _ in range(num_dense_layers):
        model.add(Dense(units=units, activation='relu'))
        model.add(Dropout(rate))
        
    model.add(Dense(1, activation='sigmoid'))  # Assuming binary classification
    
    model.compile(optimizer=Adam(learning_rate = lr), loss='binary_crossentropy', 
                  metrics=['accuracy', 'recall', 'Precision', 'FalseNegatives'])
    return model

In [38]:
from sklearn.model_selection import cross_validate
batch_size=26
epochs=30
filters=44
kernel_size=4
lr=0.0017702277771397554
num_conv_layers=1
num_dense_layers=1
pool_size= 2
rate=0.5
units=94
scoring = ['accuracy', 'recall',  'precision','f1_macro', 'f1_weighted']
def create_model():
    return cnn_model(lr, filters, kernel_size, pool_size,num_conv_layers, 
              num_dense_layers, rate, units)
# Define KerasClassifier with build_fn
model = KerasClassifier(build_fn=create_model, epochs=epochs, batch_size=batch_size, verbose=0)
# Perform cross-validation
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=123)
# score = cross_val_score(model, X_train, y_train, scoring=scoring, cv=kfold)
scores = cross_validate(model, X_train, y_train, scoring=scoring, cv=kfold)
cnn_cv_scores = pd.DataFrame(scores)
display(cnn_cv_scores)

,fit_time,score_time,test_accuracy,test_recall,test_precision,test_f1_macro,test_f1_weighted
0,108.548832,0.114046,0.812903,0.651899,0.762963,0.783247,0.808938
1,104.454866,0.115286,0.818966,0.694268,0.751724,0.793835,0.817104
2,105.093937,0.115513,0.842672,0.777070,0.762500,0.825120,0.843031
3,104.158407,0.115311,0.851293,0.751592,0.797297,0.831508,0.850173
4,109.225554,0.115118,0.844828,0.683544,0.830769,0.818750,0.840679
5,105.807852,0.120733,0.834052,0.708861,0.783217,0.810690,0.831902
6,108.070742,0.113466,0.870690,0.810127,0.810127,0.856044,0.870690
7,107.549362,0.114945,0.829741,0.803797,0.725714,0.814995,0.831655
8,110.905580,0.114803,0.840517,0.797468,0.750000,0.825041,0.841639
9,104.537937,0.113840,0.851293,0.860759,0.743169,0.840054,0.853578


In [45]:
cnn_cv_scores.to_csv('cv.csv', index=False)

In [39]:
avg_f1_macro_cv_score = cnn_cv_scores['test_f1_macro'].mean()
avg_f1_weighted_cv_score = cnn_cv_scores['test_f1_weighted'].mean()
avg_precision_cv_score = cnn_cv_scores['test_precision'].mean()
avg_recall_cv_score = cnn_cv_scores['test_recall'].mean()
avg_acc_cv_score = cnn_cv_scores['test_accuracy'].mean()

acc_perc = round((avg_acc_cv_score * 100), 2)
recall_perc = round((avg_recall_cv_score * 100), 2)
precision_perc = round((avg_precision_cv_score * 100), 2)
weighted_perc = round((avg_f1_weighted_cv_score * 100), 2)
macro_perc = round((avg_f1_macro_cv_score * 100), 2)

print(f'Precision: {avg_precision_cv_score}', f'| Prec %: {precision_perc}')
print(f'Recall: {avg_recall_cv_score}', f' | Recall %: {recall_perc}')
print(f'Accuracy: {avg_acc_cv_score}', f' | Accuracy %: {acc_perc}')
print(f'Macro F1: {avg_f1_macro_cv_score}', f' | Macro F1 %: {macro_perc}')
print(f'Weighted F1: {avg_f1_weighted_cv_score}', f' | Weighted F1 %: {weighted_perc}')

Precision: 0.771748067907718 | Prec %: 77.17
Recall: 0.7539385632508264  | Recall %: 75.39
Accuracy: 0.8396954949944384  | Accuracy %: 83.97
Macro F1: 0.8199282284817473  | Macro F1 %: 81.99
Weighted F1: 0.8389387016941903  | Weighted F1 %: 83.89


In [41]:
history = model.fit(X_train, y_train, epochs=30,batch_size=26)

In [44]:
from sklearn.metrics import make_scorer, accuracy_score, recall_score, precision_score
y_test_predict = model.predict(X_test)

test_accuracy = accuracy_score(y_test, y_test_predict)
test_recall = recall_score(y_test, y_test_predict)
test_precision = precision_score(y_test, y_test_predict)

test_acc_perc = round((test_accuracy * 100), 2)
test_recall_perc = round((test_recall * 100), 2)
test_precision_perc = round((test_precision * 100), 2)

print(f'Test Precision: {test_precision}', f" | Test Set Accuracy %: {test_acc_perc}")
print(f'Training Set Recall: {test_recall}', f' | Test Set Recall %: {test_recall_perc}')
print(f"Training Set Accuracy: {test_accuracy}", f' | Test Set Precision %: {test_precision_perc}')

from sklearn.metrics import f1_score
macrof1_score = f1_score(y_test, y_test_predict, average='macro')
macro_perc = round((macrof1_score * 100), 2)
print(f'Macro F1: {macrof1_score}',f' | Test Macro F1 %: {macro_perc}')

wf1_score = f1_score(y_test, y_test_predict, average='weighted')
weighted_perc = round((wf1_score * 100), 2)
print(f'Weighted F1: {wf1_score}',f' | Test Weighted F1 %: {weighted_perc}')

Test Precision: 0.7617866004962779  | Test Set Accuracy %: 84.15
Training Set Recall: 0.7772151898734178  | Test Set Recall %: 77.72
Training Set Accuracy: 0.8415159345391904  | Test Set Precision %: 76.18
Macro F1: 0.8243443253803802  | Test Macro F1 %: 82.43
Weighted F1: 0.8418943722066021  | Test Weighted F1 %: 84.19


In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score
dtree_model = DecisionTreeClassifier(max_depth=10, random_state=42) # Initialize a decision tree regressor
scoring = ['accuracy', 'recall',  'precision','f1_macro', 'f1_weighted' ]
scores = cross_validate(dtree_model, X_train, y_train, scoring=scoring, cv=10)
dtree_cv_scores = pd.DataFrame(scores)
display(dtree_cv_scores)

In [33]:
batch_size=26
epochs=30
filters=44
kernel_size=4
lr=0.0017702277771397554
num_conv_layers=1
num_dense_layers=1
pool_size= 2
rate=0.5
units=94
model = cnn_model(lr, filters, kernel_size, pool_size,num_conv_layers, 
              num_dense_layers, rate, units)

history = model.fit(X_train, y_train,validation_split=0.2, epochs=30,batch_size=26)

Epoch 1/30
143/143 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - FalseNegatives: 306.7222 - Precision: 0.6674 - accuracy: 0.7428 - loss: 0.5458 - recall: 0.4684 - val_FalseNegatives: 102.0000 - val_Precision: 0.7655 - val_accuracy: 0.8127 - val_loss: 0.4177 - val_recall: 0.6973
Epoch 2/30
143/143 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - FalseNegatives: 214.7431 - Precision: 0.7432 - accuracy: 0.8133 - loss: 0.4184 - recall: 0.6620 - val_FalseNegatives: 106.0000 - val_Precision: 0.7938 - val_accuracy: 0.8213 - val_loss: 0.4103 - val_recall: 0.6855
Epoch 3/30
143/143 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - FalseNegatives: 183.0417 - Precision: 0.7640 - accuracy: 0.8316 - loss: 0.3743 - recall: 0.6967 - val_FalseNegatives: 81.0000 - val_Precision: 0.7552 - val_accuracy: 0.8235 - val_loss: 0.3976 - val_recall: 0.7596
Epoch 4/30
143/143 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - FalseNegatives: 182.3056 - Precision: 0.7702 - accuracy: 0.8255 - loss: 0.3838 - recall: 0.6976 - val_FalseNegatives: 87.0000 - val_Precisi

In [32]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

# Define the grid search parameters
lr = [0.001, 0.01]
batch_size = [16, 32]
epochs = [10, 20, 30]
filters = [32, 64]
kernel_size = [3, 5]
pool_size = [2, 3]
num_conv_layers = [1, 2, 3]
num_dense_layers = [1, 2, 3]
rate = [0.3, 0.4, 0.5]
units = [64, 128]

# Define KerasClassifier with build_fn
model = KerasClassifier(build_fn=cnn_model, verbose=0, lr = [0.001, 0.01], filters = [32, 64],
                        kernel_size = [3, 5], pool_size = [2, 3], num_conv_layers = [1, 2, 3], 
                        num_dense_layers = [1, 2, 3], rate = [0.3, 0.4, 0.5], units = [64, 128])



# Define the grid search parameters
param_grid = dict(
    lr=lr,
    batch_size=batch_size,
    epochs=epochs,
    filters=filters,
    kernel_size=kernel_size,
    pool_size=pool_size,
    num_conv_layers=num_conv_layers,
    num_dense_layers=num_dense_layers,
    rate=rate,
    units=units
)
# Setup GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=10, verbose=1)
grid.fit(X_train, y_train)


Fitting 10 folds for each of 5184 candidates, totalling 51840 fits


KeyboardInterrupt: 

In [98]:
param_grid = {
    'build_fn__lr': [0.0001, 0.001, 0.01, 0.1],
    'batch_size': [16, 32],
    'epochs': [10, 20, 30, 40 , 50],
    'build_fn__filters': [32, 64,128],
    'build_fn__kernel_size': [3, 5],
    'build_fn__pool_size': [2, 3],
    'build_fn__num_conv_layers': [1, 2, 3],  # Number of Conv1D layers to test
    'build_fn__num_dense_layers': [1, 2, 3],  # Number of Dense layers to test
    'build_fn__rate': [0.2, 0.3, 0.4, 0.5],  # Dropout rates to test
    'build_fn__units': [64, 128, 256]
}

In [94]:
from sklearn.model_selection import GridSearchCV
# Create the KerasClassifier wrapper


grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=10, verbose=1)
grid.fit(X_train, y_train)

Fitting 10 folds for each of 51840 candidates, totalling 518400 fits


AttributeError: 'function' object has no attribute 'set_params'

In [27]:
def model_def(neurons, activation, learning_rate, batch_size, 
              epochs, conv_layers, conv_filters, conv_kernel_size,
             dense_layers, dropout, normalization, dropout_rate):
    # Round numerical parameters to the nearest int
    neurons = int(round(neurons))
    batch_size = int(round(batch_size))
    epochs = int(round(epochs))
    conv_layers = int(round(conv_layers))
    conv_filters = int(round(conv_filters))
    print(conv_filters)
    conv_kernel_size = int(round(conv_kernel_size))
    dense_layers = int(round(dense_layers))

    def create_model():
        ## create model by passing the paramters
        return cnn_model(neurons, activation, learning_rate, batch_size,epochs,
                      conv_layers, conv_filters, conv_kernel_size, 
                      dense_layers, normalization, dropout, dropout_rate)
    
    return KerasClassifier(create_model, batch_size=batch_size, epochs=epochs)


In [28]:
from scikeras.wrappers import KerasClassifier

In [80]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'max_depth': [10, 20, 30, None]
}
scoring = ['accuracy', 'recall',  'precision','f1_macro', 'f1_weighted' ]
# Create KerasClassifier
model = KerasClassifier(build_fn=cnn_model, verbose=0)
grid_search = GridSearchCV(estimator=model, param_grid=params_cnn, cv=10, verbose=0, scoring='accuracy')
result = grid_search.fit(X_train, y_train)

NameError: name 'params_cnn' is not defined

## Miscellenious

In [23]:
from tensorflow.keras.layers import Layer, Input

In [24]:
# Load ELMo model from TensorFlow Hub
elmo = hub.load("https://tfhub.dev/google/elmo/3")

In [25]:
# Custom layer to handle ELMo embeddings
def ElmoEmbedding(x):
    def elmo_vectors(text):
        return elmo.signatures['default'](tf.constant([text]))['elmo']

    return tf.squeeze(elmo_vectors(tf.squeeze(tf.cast(x, tf.string))))

In [26]:
import tensorflow_text as text

In [27]:
def preprocess_text(text_list):
    embeddings = elmo_embeddings(text_list)
    # Flatten embeddings if needed
    embeddings_np = np.array([np.mean(embed.numpy(), axis=0) for embed in embeddings])
    return embeddings_np

In [28]:
def text_num_cnn(neurons, activation, optimizer, learning_rate, batch_size, epochs,
                conv_layers, conv_filters, conv_kernel_size,
                dense_layers, normalization, dropout, dropout_rate):
    ## list and dictionary of several optimizers
    optimizerL = ['SGD','Adam','RMSprop','Adadelta','Adagrad','Adamax','Nadam','Ftrl']
    optimizerD = {'SGD':SGD(learning_rate = learning_rate),
                  'Adam':Adam(learning_rate = learning_rate),
                  'RMSprop':RMSprop(learning_rate = learning_rate),
                  'Adadelta':Adadelta(learning_rate = learning_rate),
                  'Adagrad':Adagrad(learning_rate = learning_rate),
                  'Adamax':Adamax(learning_rate = learning_rate),
                  'Nadam':Nadam(learning_rate = learning_rate),
                  'Ftrl':Ftrl(learning_rate = learning_rate)}

    ## list of activation function
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential']


    #set variables by getting the next int 
    neurons = round(neurons)
    # this is to get the activation function being used or optimizer being used for
    # a given iteration of parameter se;ection
    activation = activationL[round(activation)]
    optimizer = optimizerL[round(optimizer)]
    batch_size = round(batch_size)
    learning_rate = round(learning_rate)
    epochs = round(epochs)
    conv_layers = round(conv_layers)
    conv_filters = round(conv_filters)
    conv_kernel_size = round(conv_kernel_size)
    dense_layers = round(dense_layers)

    ## build model architecture
    ### Textual model
    #### Define a text input layer that takes in a string of text
    input_text = Input(shape=(1,), dtype=tf.string, name='input_text')
    print(f"Input text shape: {input_text.shape}")
    print(f'Type: {input_text}')
    
    # Obtain ELMo embeddings
    # ELMo embedding layer
    embedding = Lambda(ElmoEmbedding, output_shape=(1024, ))(input_text)
    # elmo_model = hub.KerasLayer("https://tfhub.dev/google/elmo/3", trainable=False)
    # embedding = elmo_model(input_text)["default"]
    # print(f"elmo shape: {embedding.shape}")
    # embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length)(input_text)
        
    # Flatten ELMo embeddings (if necessary)
    x_text = Flatten()(embedding)
    # print(f"x_text shape after Flatten: {x_text.shape}")
    
    ## build numerical model
    x_numeric = Input(shape = (10, ), name="input_num")

    ## COncatenate the numerical and text models
    x = Concatenate()([x_text, x_numeric])
    print(f"Concatenated model: {x.shape}")
    ## Reshape to fit Conv1D input
    x = Reshape((x.shape[1], 1))(x)
    print(f"Concatenated model after reshape: {x.shape}")
    
    ## Define Convolutional Layers
    for i in range(conv_layers):
        ### create a 1D-conv layers with given number of filters and kernel sizes and a given activation function 
        ### from the function list with input staring from the concatenated neurons
        x = Conv1D(filters=conv_filters, kernel_size=conv_kernel_size, activation=activation)(x)
        print(f"Concatenated model after iteration {i+1}: {x.shape}")
        if normalization > 0.5:
            x = BatchNormalization()(x)
    ## apply pooling to the given output x
    x = GlobalMaxPooling1D()(x)
    print(f"Shape after GlobalMaxPooling1D layer {i + 1}: {x.shape}")
    ## flatten the layer 
    x = Flatten()(x)

    ## Define dense layers
    for i in range(dense_layers):
        ### define dense layer with the given amout of neurons and activation function
        x = Dense(neurons, activation=activation)(x)

        if dropout>0.5:
            x = Dropout(dropout_rate, seed=123)()

    ## Define an output layer where the final prediction is made
    output = Dense(1, activation='sigmoid')(x)
    ## Define the model
    model = Model(inputs=[x_text, x_numeric], outputs = output)
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy', 'recall', 'Precision', 'FalseNegatives'])


    ## Add early stopping to prevent model from continuing to find optimal parameters after the 20th
    es = EarlyStopping(monitor='accuracy', mode = 'max', verbose=0, patience=20)
    nn = KerasClassifier(build_fn =model, epochs=epochs, batch_size=batch_size, verbose=0)
    kfold = StratifiedKFold(n_splits =5, shuffle=True, random_state=123)
    score = cross_val_score(nn, [X_train_text, train_norm_num_features], y_train, scoring='accuracy')
    return -score

# Optimization of CNN model

In [20]:
score_acc = make_scorer(accuracy_score)

In [25]:
    def cnn_model(neurons, activation, optimizer, learning_rate, batch_size,epochs,
                  conv_layers, conv_filters, conv_kernel_size, 
                  dense_layers, normalization, dropout, dropout_rate):

        optimizer_instance = Adam(learning_rate = learning_rate)
        ## build model architecture
        input_x = Input(shape=(1034,))

        ## Reshape to fit Conv1D input
        x = Reshape((input_x.shape[1], 1))(input_x)

        ## Define Convolutional Layers
        for i in range(conv_layers):
            ### create a 1D-conv layers with given number of filters and kernel sizes and a given activation function 
            ### from the function list with input staring from the concatenated neurons
            x = Conv1D(filters=conv_filters, kernel_size=conv_kernel_size, activation=activation)(x)

            if normalization > 0.5:
                x = BatchNormalization()(x)
                
        ## apply pooling to the given output x
        x = GlobalMaxPooling1D()(x)

        ## flatten the layer 
        x = Flatten()(x)
        
        ## Define dense layers
        for i in range(dense_layers):
            ### define dense layer with the given amout of neurons and activation function
            x = Dense(neurons, activation=activation)(x)
    
            if dropout>0.5:
                x = Dropout(dropout_rate, seed=123)(x)

        ## Define an output layer where the final prediction is made
        output = Dense(1, activation='sigmoid')(x)
        
        ## Define the model
        model = Model(inputs=input_x, outputs = output)
        model.compile(loss='binary_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy', 'recall', 'Precision', 'FalseNegatives'])
        return model

In [22]:
def text_num_cnn(neurons, activation, optimizer, learning_rate, batch_size, epochs,
                 conv_layers, conv_filters, conv_kernel_size,
                 dense_layers, normalization, dropout, dropout_rate):
    
    # Define lists and dictionaries for optimizers and activation functions
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential']
    optimizerL = ['SGD','Adam','RMSprop','Adadelta','Adagrad','Adamax','Nadam','Ftrl']
    optimizerD = {'SGD':SGD(learning_rate = learning_rate),
                  'Adam':Adam(learning_rate = learning_rate),
                  'RMSprop':RMSprop(learning_rate = learning_rate),
                  'Adadelta':Adadelta(learning_rate = learning_rate),
                  'Adagrad':Adagrad(learning_rate = learning_rate),
                  'Adamax':Adamax(learning_rate = learning_rate),
                  'Nadam':Nadam(learning_rate = learning_rate),
                  'Ftrl':Ftrl(learning_rate = learning_rate)}

    
    # Convert indices to corresponding names and instantiate optimizer
    activation = activationL[int(round(activation))]
    optimizer_name = optimizerL[int(round(optimizer))]
    optimizer_instance = Adam(learning_rate = learning_rate)
    
    # Round numerical parameters
    neurons = int(round(neurons))
    batch_size = int(round(batch_size))
    epochs = int(round(epochs))
    conv_layers = int(round(conv_layers))
    conv_filters = int(round(conv_filters))
    conv_kernel_size = int(round(conv_kernel_size))
    dense_layers = int(round(dense_layers))
    
    # Define the Keras model with the provided parameters
    def create_model():
        return cnn_model(
            neurons=neurons,
            activation=activation,
            optimizer=optimizer_instance,
            learning_rate=learning_rate,
            batch_size=batch_size,
            epochs=epochs,
            conv_layers=conv_layers,
            conv_filters=conv_filters,
            conv_kernel_size=conv_kernel_size,
            dense_layers=dense_layers,
            normalization=normalization,
            dropout=dropout,
            dropout_rate=dropout_rate
        )
    
    # Create KerasClassifier
    model = KerasClassifier(build_fn=create_model, epochs=epochs, batch_size=batch_size, verbose=0)
    
    # Define early stopping
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=0, patience=20)
    
    # Perform cross-validation
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=kfold)
    
    score=np.nan_to_num(score)

    score=score.mean()
    return score


## Define the parameters 

In [23]:
params_cnn = {
    'neurons': (10, 100),           # Number of neurons in dense layers
    'activation': (0, 7),           # Type of activation function
    'optimizer': (0, 7),            # Optimization algorithm
    'learning_rate': (0.001, 0.1),  # How fast the model learns
    'batch_size': (16, 128),        # Number of samples used in each training step
    'epochs': (10, 50),             # Number of times the model sees the entire dataset
    'conv_layers': (1, 4),          # Number of layers that find patterns in data
    'conv_filters': (32, 128),      # Number of different patterns each layer looks for
    'conv_kernel_size': (3, 5),     # Size of the window that moves over the data
    'dense_layers': (1, 3),         # Layers that process information after finding patterns
    'dropout': (0.0, 0.5),
    'normalization': (0.0, 1.0), # Technique to prevent overfitting
    'dropout_rate': (0.1, 0.5)      # How much information is randomly ignored to prevent overfitting
}

In [69]:
print(f'Shape of X_train_norm: {X_train.shape}')
print(f'Shape of y_train: {y_train.shape}')

Shape of X_train_norm: (4641, 1034)
Shape of y_train: (4641,)


In [27]:
# Initialize the optimizer
cnn_bo_optimizer = BayesianOptimization(text_num_cnn, params_cnn, random_state=111)

# Run the optimizer
cnn_bo_optimizer.maximize(n_iter=100)

# # Print the best result
# print(cnn_bo_optimizer.max)

|   iter    |  target   | activa... | batch_... | conv_f... | conv_k... | conv_l... | dense_... |  dropout  | dropou... |  epochs   | learni... |  neurons  | normal... | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
| 1         | 0.132     | 4.285     | 34.94     | 73.86     | 4.539     | 1.886     | 1.298     | 0.01124   | 0.2681    | 19.55     | 0.03443   | 99.16     | 0.2377    | 0.5683    |
| 2         | 0.1059    | 4.687     | 85.58     | 58.33     | 3.932     | 1.355     | 1.148     | 0.4504    | 0.4176    | 43.62     | 0.08171   | 99.19     | 0.5773    | 5.696     |
| 3         | 0.132     | 2.949     | 19.07     | 75.6      | 3.211     | 3.452     | 2.395     | 0.2826    | 0.2097    | 49.94     | 0.01467   | 65.39     | 0.485     | 2.835     |
| 4         | 0.132     | 5.095     | 52.16     | 70.45     | 3.638     | 3.842     | 2.83

| 18        | 0.1369    | 7.0       | 48.19     | 128.0     | 4.621     | 1.0       | 1.0       | 0.5       | 0.3947    | 50.0      | 0.001     | 27.21     | 0.2302    | 1.411     |
| 19        | 0.1457    | 0.0       | 71.84     | 128.0     | 3.0       | 4.0       | 3.0       | 0.0       | 0.5       | 50.0      | 0.05362   | 45.75     | 1.0       | 7.0       |
| 20        | 0.1466    | 0.0       | 80.22     | 128.0     | 3.0       | 4.0       | 3.0       | 0.5       | 0.5       | 50.0      | 0.001     | 62.46     | 1.0       | 0.0       |
| 21        | 0.132     | 7.0       | 83.27     | 112.4     | 3.0       | 1.0       | 3.0       | 0.5       | 0.5       | 50.0      | 0.1       | 51.89     | 1.0       | 5.484     |
| 22        | 0.132     | 3.61      | 48.16     | 35.41     | 3.337     | 3.054     | 2.039     | 0.4497    | 0.1051    | 36.51     | 0.04057   | 23.44     | 0.793     | 6.531     |
| 23        | 0.1539    | 0.0       | 62.43     | 128.0     | 3.0       | 4.0       | 3.0 

| 67        | 0.132     | 1.986     | 23.44     | 106.4     | 4.799     | 3.711     | 1.252     | 0.04829   | 0.4682    | 28.08     | 0.06447   | 29.05     | 0.1311    | 2.992     |
| 68        | 0.132     | 5.953     | 27.17     | 58.22     | 3.761     | 1.864     | 1.72      | 0.005637  | 0.4083    | 44.68     | 0.03107   | 54.27     | 0.6094    | 0.3428    |
| 69        | 0.132     | 3.841     | 65.18     | 102.9     | 3.572     | 1.275     | 2.774     | 0.2704    | 0.2586    | 48.73     | 0.02801   | 56.57     | 0.7781    | 2.748     |
| 70        | 0.132     | 6.539     | 80.41     | 111.9     | 3.898     | 3.749     | 1.515     | 0.4632    | 0.4537    | 42.01     | 0.0314    | 67.71     | 0.847     | 2.298     |
| 71        | 0.132     | 1.228     | 114.7     | 99.73     | 4.891     | 2.62      | 2.664     | 0.3855    | 0.4503    | 20.99     | 0.07016   | 55.28     | 0.3847    | 5.244     |
| 72        | 0.132     | 3.013     | 49.03     | 79.7      | 3.299     | 2.84      | 2.45

In [28]:
# # Print the best result
print(cnn_bo_optimizer.max)

{'target': 0.15715823466092574, 'params': {'activation': 6.144992430691681, 'batch_size': 41.108919894277264, 'conv_filters': 110.20823858391267, 'conv_kernel_size': 4.681361698460874, 'conv_layers': 3.9953514753525634, 'dense_layers': 2.4804188434742707, 'dropout': 0.009722558954651683, 'dropout_rate': 0.10735259626754301, 'epochs': 24.391264529856464, 'learning_rate': 0.004333651471070642, 'neurons': 83.07749075522084, 'normalization': 0.10104973789839489, 'optimizer': 4.764720919234766}}


In [29]:
# Extract the results
results = cnn_bo_optimizer.res
# Create a DataFrame from the results
results_df = pd.DataFrame(results)
# Optionally, save the DataFrame to a CSV file
results_df.to_csv('cnn_bayesian_optimization_results.csv', index=False)

In [31]:
import json
# Extract the best result
best_result = cnn_bo_optimizer.max

# Save the best result to a JSON file
with open('cnn_best_result.json', 'w') as f:
    json.dump(best_result, f)

In [44]:
print(int(round(6.144992430691681)))

6


# Model Test with Optimal Parameters

In [33]:
def plot_all_metrics(history):
    metrics = ['accuracy', 'loss', 'recall', 'Precision', 'FalseNegatives']
    
    plt.figure(figsize=(15, 10))

    for i, metric in enumerate(metrics, 1):
        plt.subplot(2, 3, i)
        plt.plot(history.history[metric], label='Training ' + metric)
        plt.plot(history.history['val_' + metric], label='Validation ' + metric)
        plt.xlabel('Epochs')
        plt.ylabel(metric.capitalize())
        plt.legend()
        plt.title(metric.capitalize() + ' over Epochs')
        plt.tight_layout()

    plt.show()

In [68]:
neurons = 83.07749075522084
activation= 6.144992430691681
batch_size = 41.108919894277264
conv_filters = 110.20823858391267
conv_kernel_size = 4.681361698460874
conv_layers = 3.9953514753525634
dense_layers = 2.4804188434742707
dropout = 0.009722558954651683
normalization = 0.10104973789839489
learning_rate = 0.004333651471070642
optimizer = Adam(learning_rate = learning_rate)
epochs = 24.391264529856464
dropout_rate = 0.10735259626754301

activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential']
activation = activationL[int(round(activation))]

# Round numerical parameters
neurons = int(round(neurons))
batch_size = int(round(batch_size))
epochs = int(round(epochs))
conv_layers = int(round(conv_layers))
conv_filters = int(round(conv_filters))
conv_kernel_size = int(round(conv_kernel_size))
dense_layers = int(round(dense_layers))

model = cnn_model(
            neurons=neurons,
            activation=activation,
            optimizer=optimizer,
            learning_rate=learning_rate,
            batch_size=batch_size,
            epochs=epochs,
            conv_layers=conv_layers,
            conv_filters=conv_filters,
            conv_kernel_size=conv_kernel_size,
            dense_layers=dense_layers,
            normalization=normalization,
            dropout=dropout,
            dropout_rate=dropout_rate
        )
history = model.fit(X_train, y_train, validation_split = 0.2, batch_size = batch_size, epochs = epochs)

# Create KerasClassifier
keras_clf = KerasClassifier(build_fn=model,optimizer=optimizer, batch_size = batch_size, epochs = epochs, verbose=0)

# Perform cross-validation
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
score = cross_val_score(keras_clf, X_train, y_train, scoring='accuracy', cv=kfold)
score = np.nan_to_num(score)
cv_score = pd.Dataframe(score)
display(cv_score)
# history = model.fit(X_train, y_train, validation_split = 0.2, batch_size = batch_size, epochs = epochs)
# plot_all_metrics(history)

Epoch 1/24
91/91 ━━━━━━━━━━━━━━━━━━━━ 12s 120ms/step - FalseNegatives: 538.8587 - Precision: 0.3681 - accuracy: 0.6045 - loss: 0.7129 - recall: 0.2047 - val_FalseNegatives: 337.0000 - val_Precision: 0.0000e+00 - val_accuracy: 0.6372 - val_loss: 0.6372 - val_recall: 0.0000e+00
Epoch 2/24
91/91 ━━━━━━━━━━━━━━━━━━━━ 11s 118ms/step - FalseNegatives: 506.1087 - Precision: 0.6084 - accuracy: 0.6787 - loss: 0.6136 - recall: 0.1618 - val_FalseNegatives: 226.0000 - val_Precision: 0.7400 - val_accuracy: 0.7147 - val_loss: 0.5482 - val_recall: 0.3294
Epoch 3/24
91/91 ━━━━━━━━━━━━━━━━━━━━ 11s 118ms/step - FalseNegatives: 407.7717 - Precision: 0.5381 - accuracy: 0.6735 - loss: 0.6048 - recall: 0.3747 - val_FalseNegatives: 198.0000 - val_Precision: 0.7433 - val_accuracy: 0.7352 - val_loss: 0.5415 - val_recall: 0.4125
Epoch 4/24
91/91 ━━━━━━━━━━━━━━━━━━━━ 11s 118ms/step - FalseNegatives: 309.2065 - Precision: 0.6715 - accuracy: 0.7534 - loss: 0.5296 - recall: 0.5317 - val_FalseNegatives: 239.0000 - v

KeyboardInterrupt: 

## Training Evaluation

In [ ]:
train_evaluation_results = model.evaluate(X_train, y_train)
# print(f'Evaluation Results: {evaluation_results}')

In [50]:
train_evaluation_results = model.evaluate(X_train, y_train)
train_loss = train_evaluation_results[0]
train_accuracy = train_evaluation_results[1]
train_recall = train_evaluation_results[2]
train_precision = train_evaluation_results[3]
train_false_negatives = train_evaluation_results[4]

print(f'Test loss: {train_loss}')
print(f'Test accuracy: {train_accuracy}')
print(f'Test recall: {train_recall}')
print(f'Test precision: {train_precision}')
print(f'Test false negatives: {train_false_negatives}')

146/146 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - FalseNegatives: 229.2449 - Precision: 0.8410 - accuracy: 0.8592 - loss: 0.3509 - recall: 0.7114
Test loss: 0.3679976165294647
Test accuracy: 0.8474466800689697
Test recall: 0.6987951993942261
Test precision: 0.8254681825637817
Test false negatives: 475.0


In [51]:
train_acc_perc = round((train_accuracy * 100), 2)
train_recall_perc = round((train_recall * 100), 2)
train_precision_perc = round((train_precision * 100), 2)

print(f"Training Set Accuracy %: {train_acc_perc}")
print(f'Training Set Recall %: {train_recall_perc}')
print(f'Training Set Precision %: {train_precision_perc}')

Training Set Accuracy %: 84.74
Training Set Recall %: 69.88
Training Set Precision %: 82.55


In [54]:
#make prediction 
train_preds = np.round(model.predict(X_train))
print('Number of predictions made by the model: {}'.format(len(train_preds)))

146/146 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step
Number of predictions made by the model: 4641


In [55]:
from sklearn.metrics import f1_score
train_macrof1_score = f1_score(y_train, train_preds, average='macro')
train_macro_perc = round((train_macrof1_score * 100), 2)
print(train_macrof1_score)

print(f'Macro F1 %: {train_macro_perc}')

0.8228610675039246
Macro F1 %: 82.29


In [56]:
train_wf1_score = f1_score(y_train, train_preds, average='weighted')
train_weighted_perc = round((train_wf1_score * 100), 2)
print(train_wf1_score)

print(f'Macro F1 %: {train_weighted_perc}')

0.8440055396630334
Macro F1 %: 84.4


## Testing Evaluation

In [46]:
evaluation_results = model.evaluate(X_test, y_test)
print(f'Evaluation Results: {evaluation_results}')

37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - FalseNegatives: 91.8684 - Precision: 0.6903 - accuracy: 0.7747 - loss: 0.4722 - recall: 0.5654
Evaluation Results: [0.4687822759151459, 0.776055097579956, 0.5873417854309082, 0.7051671743392944, 163.0]


In [52]:
evaluation_results = model.evaluate(X_test, y_test)
test_loss = evaluation_results[0]
test_accuracy = evaluation_results[1]
test_recall = evaluation_results[2]
test_precision = evaluation_results[3]
test_false_negatives = evaluation_results[4]

print(f'Test loss: {test_loss}')
print(f'Test accuracy: {test_accuracy}')
print(f'Test recall: {test_recall}')
print(f'Test precision: {test_precision}')
print(f'Test false negatives: {test_false_negatives}')

146/146 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - FalseNegatives: 229.2449 - Precision: 0.8410 - accuracy: 0.8592 - loss: 0.3509 - recall: 0.7114
Test loss: 0.4687822759151459
Test accuracy: 0.776055097579956
Test recall: 0.5873417854309082
Test precision: 0.7051671743392944
Test false negatives: 163.0


In [53]:
test_acc_perc = round((test_accuracy * 100), 2)
test_recall_perc = round((test_recall * 100), 2)
test_precision_perc = round((test_precision * 100), 2)

print(f"Training Set Accuracy %: {train_acc_perc}")
print(f'Training Set Recall %: {train_recall_perc}')
print(f'Training Set Precision %: {train_precision_perc}')

Training Set Accuracy %: 77.61
Training Set Recall %: 58.73
Training Set Precision %: 70.52


In [58]:
#make prediction 
test_preds = np.round(model.predict(X_test))
print('Number of predictions made by the model: {}'.format(len(test_preds)))

from sklearn.metrics import f1_score
test_macrof1_score = f1_score(y_test, test_preds, average='macro')
test_macro_perc = round((test_macrof1_score * 100), 2)
print(test_macrof1_score)

print(f'Macro F1 %: {test_macro_perc}')

test_wf1_score = f1_score(y_test, test_preds, average='weighted')
test_weighted_perc = round((test_wf1_score * 100), 2)
print(test_wf1_score)

print(f'weighted F1 %: {test_weighted_perc}')

37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step
Number of predictions made by the model: 1161
0.7390902993382613
Macro F1 %: 73.91
0.7704723365935497
weighted F1 %: 77.05


# Cross Validation

In [61]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score
scoring = ['accuracy', 'recall',  'precision','f1_macro', 'f1_weighted' ]
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=kfold)
op_cv_scores = pd.DataFrame(scores)
display(op_cv_scores)

TypeError: Cannot clone object '<Functional name=functional_35, built=True>' (type <class 'keras.src.models.functional.Functional'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' method.

In [ ]:
import keras_tuner
import keras